In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
from PIL import Image

In [2]:
model=torch.hub.load('ultralytics/yolov5', 'custom', path='last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Kaiser/.cache\torch\hub\master.zip
YOLOv5  2023-10-25 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 267 layers, 46215978 parameters, 0 gradients, 108.0 GFLOPs
Adding AutoShape... 


In [3]:
import math
video_path = 'Buet.MOV'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
output_path = 'output_video.mov'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))
count = 0
prev_frame = []
tracking_object = {}
object_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    curr_frame = []
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame = frame.resize((640, 640))
    results = model(frame)
    render_frame = np.squeeze(results.render())
    render_frame = cv2.cvtColor(render_frame, cv2.COLOR_RGB2BGR)
    bounding_boxes = results.pred[0].tolist()
    for box in bounding_boxes:
        x, y, w, h = box[:4]
        cx = int((x + w) / 2)
        cy = int((y + h) / 2)
        curr_frame.append((cx, cy))
    
    if count <= 2:
        for point_1 in curr_frame:
            for point_2 in prev_frame:
                distance = math.hypot(point_2[0] - point_1[0], point_2[1] - point_1[1])
                if distance < 20:
                    tracking_object[object_id] = point_1
                    object_id += 1
    else:
        # Check whether any vehicle here with id's
        # so iterating on current frame for checking vehicle with id's
        # if there is id's then no need to update
        # if not then update
        # check whether the object exists or not also
        tracking_object_copy = tracking_object.copy()
        curr_frame_copy = curr_frame.copy()
        
        for object_id, point_2 in tracking_object_copy.items():
            object_exists = False
            for point_1 in curr_frame:
                distance = math.hypot(point_2[0] - point_1[0], point_2[1] - point_1[1])
                if distance < 20:
                    tracking_object[object_id] = point_1
                    object_exists = True
                    if point_1 in curr_frame_copy:
                        curr_frame_copy.remove(point_1)
                    continue
            if not object_exists:
                tracking_object.pop(object_id)
        
        for point_1 in curr_frame_copy:
            tracking_object[object_id] = point_1
            object_id += 1
    for object_id, pt in tracking_object.items():
        cv2.circle(render_frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(render_frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)
    
    prev_frame = curr_frame.copy()
    cv2.imshow('Object Detection', render_frame)
    out.write(render_frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
